# Description of What raw_data Outputs Look Like

In [15]:
#put into this a list of usernames to get the data for 
users_get_data_for = ["Pokechu22", "13steinj"]

#if you are using this to gather data for a group of people above set this to true. Otherwise it will scrape random users data to use for matching.
just_get_for_them = True

In [ ]:
#dataset description
"""
Redditor Instantiation
- load all data into memory first (static) 

get_base_data(): return redditor[data]
get_comment_data()
get_submission_data() 
//some kind of store value w/ state (seperate dict)
"""



"""
//all should contain stuff like a date scraped
Redditor_Dict:
Username ->  {
    "data": properties + labels in dict 
    "comments": [
        (comment date, {"subreddit_id" : subreddit_id, "submission_id": submission_id, "comment_id": comment_id})
        # (comment date, subreddit_id, .submission.id, {dictionary of all info})
        //dict of all info should be initialized seperately
    ]
}


[properties + labels below, comments: {relationship data + ["post_id" :id of post commented on [comment.subreddit_id]]}]
#what relevant edge data do we need?


SubmissionDict
{
    submission.id:  {
        "created_utc"
        "distinguished" 
        "is_original_content" 
        "over_18" 
        "score" 
        "title"
        "upvote_ratio" 
        "users_commented_on": submission.comments.replace_more(None).list() | None
        "comment_data": {"user": comment.author.id, "id": comment.id} | None
    }
}

CommentDict {
    submission_id + "--------" + comment_id : { 
        "data": {
            "is_edited": comment.edited,
            "link_title": comment.link_title, 
            "num_replies": len(comment.replies.list()),
            "score": comment.score, 
            "score_is_hidden": comment.score_hidden,
            "total_awards": comment.total_awards_received, 
            "num_ups": comment.ups, 
            "num_downs": comment.downs, 
            "body": comment.body,
            "date": comment.created_utc,
            "is_submitter": comment.is_submitter, 
            "stickied": comment.stickied
            
        },
        "meta": {
                "ITERATION_NUM": ITERATION_NUM, 
                "TIME": int(time.time()),
                "submission_id": submission_id, 
                "comment_id": comment.id,
                "author": redditor.name
        }
    }

}

"""


# Code

In [1]:
import praw 
reddit = praw.Reddit(
    client_id="CL5xizTtujKZc4HOuFF_5w",
    client_secret="mD9JbzEVNFB8xopQKE_2WTcyIaz0hg",
    user_agent="logan-vaz"
)

#reddit secret mD9JbzEVNFB8xopQKE_2WTcyIaz0hg, user_agen

In [3]:
ITERATION_NUM = 1
import time 

def get_redditor_data(redditor: praw.reddit):
    """get all data from a praw redditor object"""

    submission_ids = list() 

    to_ret = dict()
    to_ret["username"] = redditor.name
    to_ret["comment_karma"] = redditor.comment_karma
    # to_ret["num_comments"] = len(redditor.comments.top())
    to_ret["time_creation"] = redditor.created_utc
    to_ret["verified_email"] = redditor.has_verified_email
    to_ret["is_employee"] = redditor.is_employee
    to_ret["is_mod"] = redditor.is_mod
    to_ret["is_gold"] = redditor.is_gold 
    to_ret["is_suspended"] = False
    to_ret["link_karma"] = redditor.link_karma
    to_ret["num_moderated"] = len(redditor.moderated())
    to_ret["num_multireddits"] = len(redditor.multireddits())
    to_ret["num_trophies"] = len(redditor.trophies())

    if (redditor.subreddit):
        to_ret["has_subreddit"] = True
        to_ret["over_18"] = redditor.subreddit.over_18#redditor.subreddit["over_18"]
        to_ret["num_subscribers"] = redditor.subreddit.subscribers#redditor.subreddit["subscribers"]

        #this is the only one added that rlly shouldn't be
        to_ret["public_description"] = redditor.subreddit.public_description
    else: to_ret["has_subreddit"] = False  
    
    to_ret = {"data": to_ret} 

    comment_data_user = list() 
    comment_data_dict = dict()

    comment_cnt = 0 
    for comment in redditor.comments.new():
        print("comment is", comment_cnt)
        comment_cnt += 1
        submission_id = comment.link_id 
        submission_ids.append(submission_id)
        submission_id =comment.submission.id

        # print("Created date is " , comment.created_utc)
        comment_data_user.append([int(comment.created_utc), {"subreddit_id" : comment.subreddit_id, "submission_id": submission_id, "comment_id": comment.id}])

        comment_key = submission_id + "--------" + comment.id
        comment_data = {

            "data": {
                "is_edited": comment.edited,
                "link_title": comment.link_title, 
                "num_replies": len(comment.replies.list()),
                "score": comment.score, 
                "score_is_hidden": comment.score_hidden,
                "total_awards": comment.total_awards_received, 
                "num_ups": comment.ups, 
                "num_downs": comment.downs, 
                "body": comment.body,
                "date": comment.created_utc,
                "is_submitter": comment.is_submitter, 
                "stickied": comment.stickied
                
            },
            "meta": {
                 "ITERATION_NUM": ITERATION_NUM, 
                 "TIME": int(time.time()),
                 "submission_id": submission_id, 
                 "comment_id": comment.id,
                 "author": redditor.name
            }
            
        }
        comment_data_dict[comment_key] = comment_data
    
    to_ret["comments"] = comment_data_user 
    to_ret["meta"] = {
        "ITERATION_NUM": ITERATION_NUM, 
        "TIME": int(time.time())
    }
    print("returning triplet")
    return to_ret, comment_data_dict, submission_ids



# def get_submission_data():
    

In [4]:
import prawcore #required for exception handling

In [6]:
def get_comments(listed):
    """from a list of morecomments and comments extract comments, implements backoff"""
    to_ret = list()
    for l in listed:
        if (isinstance(l, praw.models.MoreComments)):
            time.sleep(0.05)
            try:
                
                base_comments = get_comments(l.comments())
            except prawcore.exceptions.TooManyRequests as e:
                print("backoff 1")
                time.sleep(10)
                try:
                    base_comments = get_comments(l.comments())
                except prawcore.exceptions.TooManyRequests as e:
                    print("backoff 2")
                    time.sleep(30)
                    base_comments = get_comments(l.comments())
            to_ret += base_comments 
        else:
            to_ret.append(l)
    return to_ret

In [7]:
#https://stackoverflow.com/questions/842557/how-to-prevent-a-block-of-code-from-being-interrupted-by-keyboardinterrupt-in-py
#interrupt handler
import signal
import logging

class DelayedKeyboardInterrupt:

    def __enter__(self):
        self.signal_received = False
        self.old_handler = signal.signal(signal.SIGINT, self.handler)
                
    def handler(self, sig, frame):
        self.signal_received = (sig, frame)
        logging.debug('SIGINT received. Delaying KeyboardInterrupt.')
    
    def __exit__(self, type, value, traceback):
        signal.signal(signal.SIGINT, self.old_handler)
        if self.signal_received:
            self.old_handler(*self.signal_received)


new_authors = list()

In [8]:
# hottest_submissions = list()
# for submission in reddit.subreddit("depression").hot():
#     # print(dir(submission))
#     # print("subreddit_subscribers", submission.subreddit_subscribers)
#     # print("comments ", submission.comments)
#     # print("author_fullname", submission.author_fullname)
#     # print("author", submission.author)

#     # break
#     hottest_submissions.append(submission)

# new_authors = [h.author.name for h in hottest_submissions]

In [ ]:
import json 
import os
REDDITOR_FILE = "raw_data/redditor_dict.json"
SUBMISSION_FILE = "raw_data/submission_dict.json"
COMMENT_FILE = "raw_data/comment_dict.json"

def save_get_json(file_name):
    if (os.path.exists(file_name)):
        with open(file_name, 'r') as f:
            return json.load(f)
    return dict()


redditor_dict = save_get_json(REDDITOR_FILE)
submission_dict = save_get_json(SUBMISSION_FILE)
comment_dict = save_get_json(COMMENT_FILE)

print("heads up this takes a while")

print("len of redditor_dict is ", len(redditor_dict))
print("len of submission_dict is ", len(submission_dict))
print("len of comment_dict is ", len(comment_dict))

In [16]:
import traceback
import time    
import random
from atomicwrites import atomic_write

import numpy as np 
epoch_time = int(time.time())

time.sleep(1)

#first, let's just fill with the treatment group - by name
treatment_group = ['MoosieGoose', 'JollyK9', 'Southern_Ad3032', 'bduwowy272habbw', 'Late_Introduction203', 'kapster68', 'TheApertureMonkey', 'talemoon22', 'sebagolindenwald', 'spicyranchplzz', 'TheFloorMayBeLava_02', 'rxtten_flesh', 'greenblooded395', 'greenblooded395', 'DrakenJosh98', 'WhichUsernameIsBest', 'FStahp2', 'Pongpianskul', 'Kanashimi515', 'eviuwu', 'Kattheloner_22', 'Reeze2911', 'Sac20000', 'RanpoWasTaken', 'jlynny1811', 'Playful-Fail4778', 'GarageOk8109', 'katandcats', 'holyredemption', 'jifpeanutbutter420', 'Timely_Inflation1000', 'Erica_Peanut']

# treatment_group += new_authors
#get potential treatment groups from those that comment on at least one of the same submission
control_explore = [comment_data["user"] for submission in submission_dict.values() if submission["comment_data"] for comment_data in submission["comment_data"]]
control_explore = [c for c in control_explore if not c in redditor_dict]
random.seed(24601)
random.shuffle(control_explore)

print("length of ppl to try is ", len(control_explore))


if (just_get_for_them):
    treatment_group = list()
    control_explore = users_get_data_for


users_and_groups = [(v, "control") for v in control_explore]
users_and_groups_chunks = np.array_split([(v, "treatment") for v in treatment_group], max(int(len(treatment_group)/16), 1)) +  np.array_split(users_and_groups, max(int(len(users_and_groups)/256), 1))


#was trying to batch requests - doesn't really appear possible so we're instead doing this to hopefully avoid rate limits
for user_group_chunk in users_and_groups_chunks:
    print("chunk len is " ,len(user_group_chunk) )
    
    something_changed = False
    try:
        #get the basic user information 
        
        for user_name, group in user_group_chunk:
            time.sleep(.05)
            print("group is ", group)
            print("USER IS ", user_name)
            if (user_name in redditor_dict.keys()): 
                print("skipping")
                
                continue
            # print("continuing")
            redditor =  reddit.redditor(user_name)
            try:
                user_data, comment_data, submission_ids = get_redditor_data(redditor) 
            except AttributeError as E:
                print("Missing is ", E)
                continue

            something_changed = True

            #time to get all relevant submission data (TODO - this is how we'll add users - begging the question of the order to add them in but we'll deal w/ that later)
            this_submission_data = dict()
            cnt = 0 
            submissions_fetch = [sub_id for sub_id in submission_ids if not (sub_id in submission_dict or sub_id in this_submission_data)]
            submissions = reddit.info(fullnames=submissions_fetch)
            for submission in submissions:
                cnt += 1
                if (cnt % 10 ==0): print(f'sub cnt is {cnt}') 
                submission_id = submission.id
                if (submission_id in submission_dict or submission_id in this_submission_data): 
                    print("matched id")
                    continue 

                #get comment data for this submission - if treatment we want to extract more data
                if (group == "treatment"):
                    commentors = dict() 
                    sub_comment_data = list()
                    time.sleep(0.125/2)
                    try:
                        all_comments = submission.comments.list()
                    except prawcore.exceptions.TooManyRequests as E:
                        print("backoff 1")
                        time.sleep(10)
                        try:
                            all_comments = submission.comments.list()
                        except prawcore.exceptions.TooManyRequests as E:
                            print("backoff 2")
                            time.sleep(30)
                            all_comments = submission.comments.list()

                    original_comments = [c for c in all_comments if not isinstance(c, praw.models.MoreComments)]
                    layered_comments = get_comments([c for c in all_comments if isinstance(c, praw.models.MoreComments)])

                    for comment in original_comments:
                        if (comment.author and comment.author not in ["None", "[removed]", "[deleted]"] and comment.author.name):
                            commentors[comment.author.name] = True
                            sub_comment_data.append( {
                                "user": comment.author.name,
                                "id": comment.id,
                                "original_reply": True
                            })
                    for comment in layered_comments:
                        # submission.author != "None" or "[removed]" or "[deleted]"
                        print("comment author is ", comment.author)
                        if (comment.author and comment.author not in ["None", "[removed]", "[deleted]"] and comment.author.name):
                            commentors[comment.author.name] = True
                            sub_comment_data.append( {
                                "user": comment.author.name,
                                "id": comment.id,
                                "original_reply": False
                            })
                    this_submission_data[submission_id] = {
                        "created_utc": int(submission.created_utc),
                        "distinguished": submission.distinguished,
                        "is_original_content" : submission.is_original_content,
                        "over_18": submission.over_18,
                        "score": submission.score, 
                        "title": submission.title,
                        "edited": submission.edited, 
                        "selftext": submission.selftext,
                        "upvote_ratio": submission.upvote_ratio,
                        "users_commented_on": list(commentors.keys()),
                        "comment_data":sub_comment_data
                    }
                else:
                    this_submission_data[submission_id] = {
                        "created_utc": int(submission.created_utc),
                        "distinguished": submission.distinguished,
                        "is_original_content" : submission.is_original_content,
                        "over_18": submission.over_18,
                        "score": submission.score, 
                        "title": submission.title,
                        "edited": submission.edited, 
                        "selftext": submission.selftext,
                        "upvote_ratio": submission.upvote_ratio,
                        "users_commented_on": None,
                        "comment_data":None
                    }

            print("updating dict")
            redditor_dict.update({user_name: user_data})
            submission_dict.update(this_submission_data)
            comment_dict.update(comment_data)
    
    except KeyboardInterrupt as E:
        #if it's keyboard let's still try n save what we have
        print("trying to save what have")
        
    except Exception as E:
        traceback.print_exc()
        print("Exception is ", E)
        time.sleep(32) 
    if (something_changed):
        with DelayedKeyboardInterrupt():
            print("dont interrupt - heads up this can take a minute")
            #with open(SUBMISSION_FILE, 'w') as submission_f:
            with atomic_write(SUBMISSION_FILE, overwrite=True) as submission_f:
                json.dump(submission_dict, submission_f) 
            # with open(COMMENT_FILE, 'w') as comment_f:
            with atomic_write(COMMENT_FILE, overwrite=True) as comment_f:
                json.dump(comment_dict,comment_f )
            # with open(REDDITOR_FILE, 'w') as redditor_f:
            with atomic_write(REDDITOR_FILE, overwrite=True) as redditor_f:
                json.dump(redditor_dict,redditor_f )

        print("Saved files")
        # time.sleep(0.5)
        time.sleep(random.random()*3+.5)



"""
end
"""

length of ppl to try is  300315
chunk len is  0
chunk len is  2
group is  control
USER IS  Pokechu22
comment is 0
comment is 1
comment is 2
comment is 3
comment is 4
comment is 5
comment is 6
comment is 7
comment is 8
comment is 9
comment is 10
comment is 11
comment is 12
comment is 13
comment is 14
comment is 15
comment is 16
comment is 17
comment is 18
comment is 19
comment is 20
comment is 21
comment is 22
comment is 23
comment is 24
comment is 25
comment is 26
comment is 27
comment is 28
comment is 29
comment is 30
comment is 31
comment is 32
comment is 33
comment is 34
comment is 35
comment is 36
comment is 37
comment is 38
comment is 39
comment is 40
comment is 41
comment is 42
comment is 43
comment is 44
comment is 45
comment is 46
comment is 47
comment is 48
comment is 49
comment is 50
comment is 51
comment is 52
comment is 53
comment is 54
comment is 55
comment is 56
comment is 57
comment is 58
comment is 59
comment is 60
comment is 61
comment is 62
comment is 63
comment is 64

PermissionError: [WinError 5] Access is denied.

NameError: name 'redditor_dict' is not defined

# Ignore - No Longer an Issue If Don't Kill Kernel Correction Code - Basically Before I Implemented Atomic Rights Kernel Crashed During File Writes Casuing Problems

In [ ]:
#TODO - add code to ensure all dependencies are met bc the kernel crashed before I added atomic operations
#IE for each user check that all comments exists (and fetch them if not)
#for each comment check that all submissions exist (and fetch them if not)

In [ ]:
#NOTE - will need to rerun submissions for treatment group seperately

In [10]:
# #checking state
# comments_get = list()
# subs_get = dict()
# for user, user_data in redditor_dict.items():
#     for (_, comment) in user_data["comments"]:
#         sub_id = comment["submission_id"]
#         com_id = comment["comment_id"]
#         _key = sub_id + "--------" + com_id 
#         if not _key in comment_dict:
#             comments_get.append(_key)
#         if not sub_id in subs_get:
#             subs_get[sub_id] = True

# #checking state (each comment implies a valid submission)

# for k, v in comment_dict.items():
#     sub_id = v["meta"]["submission_id"]
#     if (not sub_id in submission_dict):
#         # subs_get.append(sub_id)
#         subs_get[sub_id] = True

# print(len(comments_get))
# print(len(subs_get))
# print(len(submission_dict))

0
734606
752455


In [11]:
# #fixing submission data bc aparently that got cut off
# def get_sub_ids(ids):
#     return [i if i.startswith('t3_') else f't3_{i}' for i in ids]

# list_of_subs = get_sub_ids(list(subs_get.keys()))

In [ ]:
# cnt = 0 
# for submission in reddit.info(list_of_subs):
#     submission_dict[submission.id] = {
#             "created_utc": int(submission.created_utc),
#             "distinguished": submission.distinguished,
#             "is_original_content" : submission.is_original_content,
#             "over_18": submission.over_18,
#             "score": submission.score, 
#             "title": submission.title,
#             "edited": submission.edited, 
#             "selftext": submission.selftext,
#             "upvote_ratio": submission.upvote_ratio,
#             "users_commented_on": None,
#             "comment_data":None
#         }
#     cnt += 1
#     if (cnt % 20 == 0): print(cnt)
#     if (cnt % 500 == 50):
#         with DelayedKeyboardInterrupt():
#             with atomic_write(SUBMISSION_FILE, overwrite=True) as submission_f:
#                 json.dump(submission_dict, submission_f) 

C:\Users\logan\AppData\Local\Temp\ipykernel_39188\508471449.py:2: DeprecationWarning: Positional arguments for 'Reddit.info' will no longer be supported in PRAW 8.
Call this function with 'fullnames' as a keyword argument.
  for submission in reddit.info(list_of_subs):


20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600


KeyboardInterrupt: 